In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:0000:0100:01


In [3]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.1.2+cpu
tiktoken version: 0.7.0


In [15]:
import urllib

# URL of the PDF file
url = "https://ag.gov.np/files/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"

# Open the URL and download the PDF
response = urllib.request.urlopen(url)
pdf_data = response.read()

# Save the PDF to a local file
with open("Constitution-of-Nepal_2072_Eng.pdf", "wb") as f:
    f.write(pdf_data)

print("PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'")


PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'


In [22]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00ta 0:00:01


In [26]:
import PyPDF2

# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()

# Print the first 500 characters to preview the content
print(raw_text[:50])


 
1 
  
 
 
 
THE CONSTITUTION OF NEPAL  
   
2 
 


In [27]:
print(len(raw_text))

344782
